In [24]:
import pandas as pd
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent))  # adds parent directory

from common.audit import AuditLogReader

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
log_dir = Path("/home/mike/workspace/soa-ilec/soa-ilec/data/workspaces/ul_model_data/")
log_rdr = AuditLogReader(log_dir)

In [ ]:
import json 

sql_log = log_rdr.traverse_sql_audit_log()
final_model_log, all_model_logs, all_by_time = res


In [60]:
from IPython.display import display, HTML
from jinja2 import Environment, FileSystemLoader, select_autoescape


# 1) Point Jinja at your templates folder (e.g., ./templates)
env = Environment(
    loader=FileSystemLoader("/home/mike/workspace/soa-ilec/soa-ilec/common/templates/"),
    autoescape=select_autoescape(["html", "xml"])
)

# 2) Load a template file (e.g., templates/page.html)
audit_template = env.get_template("audit_log.html")

# 3) Render it with context
audit_html = audit_template.render(
    sql_log = sql_log,
    cmd_log = all_by_time,    
)

display(HTML(audit_html))

In [48]:
all_by_time

[{'workspace_id': '220df849-d744-4111-b6b7-a7daac5b689d',
  'type': 1,
  'entry': None},
 {'workspace_id': 'b99aec42-9ee6-44e9-b915-0cab4f6ac449',
  'type': 2,
  'entry': {'last_workspace_id': '220df849-d744-4111-b6b7-a7daac5b689d',
   'workspace_id': 'b99aec42-9ee6-44e9-b915-0cab4f6ac449',
   'tool_name': 'cmd_create_dataset',
   'args': ['model_data_train',
    "SELECT \n  DATASET,\n  Gender,\n  COALESCE(Smoker_Status, 'UNKNOWN') AS Smoker_Status,\n  Attained_Age,\n  Number_Of_Deaths,\n  ExpDeathQx2015VBTwMI_byPol\nFROM UL_MODEL_DATA_SMALL\nWHERE DATASET='TRAIN'\n  AND ExpDeathQx2015VBTwMI_byPol > 0\n  AND Number_Of_Deaths >= 0"],
   'result': {'success': True, 'result': {'n_rows': 716.0}}}},
 {'workspace_id': '4b01ab9b-4ad8-481f-ac44-72be9e5eb1e5',
  'type': 2,
  'entry': {'last_workspace_id': 'b99aec42-9ee6-44e9-b915-0cab4f6ac449',
   'workspace_id': '4b01ab9b-4ad8-481f-ac44-72be9e5eb1e5',
   'tool_name': 'cmd_create_dataset',
   'args': ['model_data_test',
    "SELECT \n  DATASET,